# Model evaluation notebook

## Install Ollama

### Google Collab

In [ ]:
# Google Collab dependencies from: https://github.com/5aharsh/collama
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

### Docker image
An ephemeral environment can be prepared with.

```shell
docker run -v "${PWD}":/home/jovyan/work -it --rm -p 10000:8888 quay.io/jupyter/scipy-notebook:2025-03-14
```

In [ ]:
# If on Jupyter Docker image, run
docker exec -it --user=0 <docker_container_name> sh -c 'apt update && apt install -y pciutils'

In [ ]:
# Install Ollama
docker exec -it --user=0 <docker_container_name> sh -c 'curl -fsSL https://ollama.com/install.sh | sh'

## Prepare model

In [ ]:
MODEL = 'gemma2:2b'

In [ ]:
!ollama pull "$MODEL"

## Prepare code

## Run evaluation

## Cleanup

### Stop kernel

In [ ]:
get_ipython().kernel.do_shutdown(True)